# データサイエンス100本ノック（構造化データ加工編） - SAS

## はじめに
- 初めに以下のセルを実行してください
- 最初の%let でcsvファイルを置いているパスを指定してください
- 処理は複数回に分けても構いません
- ここでは例として研修中などにも使えるよう、PDVの理解が必要なコード・SQL・正規表現の使用を避けています。

- 名前、住所等はダミーデータであり、実在するものではありません
- csvのエンコーディングがutf-8のため、日本語を含む変数のlengthは長めに設定しています

In [1]:
%let dir = /home/ken-nakamatsu/100knocks ;


options compress=yes ;
title ;

data work.ds_customer ;
  infile "&dir./customer.csv" dsd missover firstobs=2;
  length customer_id $14 customer_name $40 gender_cd $1 gender $16          
         birth_day 8 age 8postal_cd $8 address $400 
         application_store_cd $6 application_date $8 status_cd $12
         ;
  informat birth_day yymmdd10. ;
  format birth_day yymmdd10. ;
  input customer_id -- status_cd ;
run ;

data work.ds_category ;
  infile "&dir./category.csv" dsd missover firstobs=2;
  length category_major_cd $2 category_major_name $100
         category_medium_cd $4 category_medium_name $100
         category_small_cd $6 category_small_name $100
        ;
  input category_major_cd -- category_small_name ;
run ;

data work.ds_product ;
  infile "&dir./product.csv" dsd missover firstobs=2;
  length product_cd $10 category_major_cd $2
         category_medium_cd $4 category_small_cd $6
         unit_price 8 unit_cost 8
         ;
  input product_cd -- unit_cost ;
run ;

data work.ds_store ;
  infile "&dir./store.csv" dsd missover firstobs=2;
  length store_cd $6 store_name $400 prefecture_cd $2 prefecture $20
         address $400 address_kana $400 tel_no $20
         longitude 8 latitude 8 floor_area 8
         ;
  input store_cd -- floor_area ;
run ;

data work.ds_receipt ;
  infile "&dir./receipt.csv" dsd missover firstobs=2;
  length sales_ymd 8 sales_epoch 8 store_cd $6 receipt_no 8
         receipt_sub_no 8 customer_id $14 product_cd $10 quantity 8 amount 8
         ;
  input sales_ymd -- amount ;
run ;

data work.ds_geocode ;
  infile "&dir./geocode.csv" dsd missover firstobs=2;
  length postal_cd  $8 prefecture $20 city $100 town $100 street $100
         address $100 full_address $300 longitude 8 latitude 8 
         ;
  input postal_cd -- latitude ;
run ;

SAS Connection established. Subprocess id is 15652


7                                                         SAS システム               2021年 3月23日 火曜日 15時44分00秒

34         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
34       ! ods graphics on / outputfmt=png;
35         
36         %let dir = /home/ken-nakamatsu/100knocks ;
37         
38         
39         options compress=yes ;
40         title ;
41         
42         data work.ds_customer ;
43           infile "&dir./customer.csv" dsd missover firstobs=2;
44           length customer_id $14 customer_name $40 gender_cd $1 gender $16
45                  birth_day 8 age 8postal_cd $8 address $400
46                  application_store_cd $6 application_date $8 status_cd $12
47                  ;
48           informat birth_day yymmdd10. ;
49           format birth_day yymmdd10. ;
50           input customer_id -- status_cd ;
51         run ;
52         
53     

# 演習問題

---
> P-001: レシート明細のデータセット（ds_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
proc print data = work.ds_receipt(obs=10) ;
run ;

Obs,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
10,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータセット（ds_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [3]:
proc print data = work.ds_receipt(obs=10) ;
  var sales_ymd customer_id product_cd amount ;
run ;

Obs,sales_ymd,customer_id,product_cd,amount
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770
10,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータセット（ds_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [4]:
proc print data = work.ds_receipt(obs=10 rename=(sales_ymd=sales_date)) ;
  var sales_date customer_id product_cd amount ;
run ;

Obs,sales_date,customer_id,product_cd,amount
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770
10,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータセット（ds_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [5]:
proc print data = work.ds_receipt ;
  var sales_ymd customer_id product_cd amount ;
  where customer_id = "CS018205000001" ;
run ;

Obs,sales_ymd,customer_id,product_cd,amount
37,20180911,CS018205000001,P071401012,2200
9844,20180414,CS018205000001,P060104007,600
21111,20170614,CS018205000001,P050206001,990
27674,20170614,CS018205000001,P060702015,108
27841,20190216,CS018205000001,P071005024,102
28758,20180414,CS018205000001,P071101002,278
39257,20190226,CS018205000001,P070902035,168
58122,20190924,CS018205000001,P060805001,495
68118,20190226,CS018205000001,P071401020,2200
72255,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータセット（ds_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [6]:
proc print data = work.ds_receipt ;
  var sales_ymd customer_id product_cd amount ;
  where customer_id = "CS018205000001" and amount >= 1000 ;
run ;

Obs,sales_ymd,customer_id,product_cd,amount
37,20180911,CS018205000001,P071401012,2200
68118,20190226,CS018205000001,P071401020,2200
72255,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データセット「ds_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:
proc print data = work.ds_receipt ;
  var sales_ymd customer_id product_cd quantity amount ;
  where customer_id = "CS018205000001" and (amount >= 1000 or quantity >= 5)  ;
run ;

Obs,sales_ymd,customer_id,product_cd,quantity,amount
37,20180911,CS018205000001,P071401012,1,2200
9844,20180414,CS018205000001,P060104007,6,600
21111,20170614,CS018205000001,P050206001,5,990
68118,20190226,CS018205000001,P071401020,1,2200
72255,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータセット（ds_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [8]:
proc print data = work.ds_receipt ;
  var sales_ymd customer_id product_cd amount ;
  where customer_id = "CS018205000001" and 1000 <= amount <= 2000 ;
run ;

Obs,sales_ymd,customer_id,product_cd,amount
72255,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータセット（ds_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [9]:
proc print data = work.ds_receipt ;
  var sales_ymd customer_id product_cd amount ;
  where customer_id = "CS018205000001" and 1000 <= amount <= 2000 ;
run ;

Obs,sales_ymd,customer_id,product_cd,amount
72255,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`ds_store where ('not(prefecture_cd == "13" | floor_area > 900)')`

In [11]:
proc print data = work.ds_store ;
  where prefecture_cd ^= "13" and floor_area <= 900 ;
run ;
/*
proc print data = work.ds_store ;
  where not(prefecture_cd = "13" or floor_area > 900) ;
run ;
*/

Obs,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
19,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831
21,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890
39,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.6612,808


---
> P-010: 店舗データセット（ds_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [12]:
proc print data = work.ds_store(obs=10) ;
  where store_cd like 'S14%' ;
run ;

Obs,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495
5,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679
8,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.5240,1548
10,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830
13,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.4630,1574
17,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341
19,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831
20,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.579,35.2964,1838
21,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890


---
> P-011: 顧客データセット（ds_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [13]:
proc print data = work.ds_customer(obs=10) ;
  where customer_id like '%1' ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
4,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
15,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
32,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
57,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
60,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
64,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
75,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
86,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
103,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データセット（ds_store）から横浜市の店舗だけ全項目表示せよ。

In [14]:
proc print data = work.ds_store ;
  where address contain '横浜市' ;
run ;

Obs,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495
8,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.5240,1548
10,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830
13,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.4630,1574
17,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341
19,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831
21,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890
27,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.576,35.5491,1657
41,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.659,35.4389,1044


---
> P-013: 顧客データセット（ds_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [15]:
proc print data = work.ds_customer(obs=10) ;
  where substr(status_cd, 1, 1) in ("A", "B", "C", "D", "E","F") ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
7,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
13,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
16,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
22,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
33,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
34,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
41,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
45,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
54,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データセット（ds_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [16]:
proc print data = work.ds_customer(obs=10) ;
  where input(scan(status_cd, -1, "-"), best.) in (1:9) ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
5,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
10,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
13,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
17,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
23,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
24,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
25,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
28,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
29,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データセット（ds_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [17]:
proc print data = work.ds_customer(obs=10) ;
  where substr(status_cd, 1, 1) in ("A", "B", "C", "D", "E","F") and input(scan(status_cd, -1, "-"), best.) in (1:9) ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
13,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
69,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
72,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
123,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
145,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
179,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
253,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
260,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
294,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
298,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データセット（ds_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [18]:
proc print data = work.ds_store ;
  where lengthn(scan(tel_no, 1, "-"))=3 and lengthn(scan(tel_no, 2, "-"))=3 and lengthn(scan(tel_no, 3, "-"))=4 ;
run ;

Obs,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.6356,1698
2,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.480,35.7057,1735
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495
5,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679
8,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.5240,1548
10,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830
12,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.438,35.5529,1087
13,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.4630,1574
17,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341


---
> P-17: 顧客データセット（ds_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [19]:
proc sort data = work.ds_customer out = work._p_17 ;
  by birth_day ;
run ;

proc print data = work._p_17(obs=10) ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
2,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
3,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
4,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
5,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
6,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
7,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
8,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
9,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0
10,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0


---
> P-18: 顧客データセット（ds_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [20]:
proc sort data = work.ds_customer out = work._p_18 ;
  by descending birth_day ;
run ;

proc print data = work._p_18(obs=10) ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
2,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
3,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
4,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
5,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
6,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
7,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
8,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
9,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
10,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データセット（ds_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [21]:
proc sort data = work.ds_receipt out = work._p_19 ;
  by descending amount ;
run ;

data work._p_19 ;
  set work._p_19(obs=10) ;
  by descending amount ;
  rank + first.amount ;
run ;

proc print data = work._p_19 ;
  var customer_id amount rank ;
run ;

Obs,customer_id,amount,rank
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,4
6,ZZ000000000000,5480,4
7,ZZ000000000000,5440,5
8,CS021515000089,5440,5
9,CS015515000083,5280,6
10,CS017414000114,5280,6


---
> P-020: レシート明細データセット（ds_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [22]:
proc sort data = work.ds_receipt out = work._p_20 ;
  by descending amount ;
run ;

data work._p_20 ;
  set work._p_20(obs=10) ;
  rank +1  ;
run ;

proc print data = work._p_20 ;
  var customer_id amount rank ;
run ;

Obs,customer_id,amount,rank
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,5
6,ZZ000000000000,5480,6
7,ZZ000000000000,5440,7
8,CS021515000089,5440,8
9,CS015515000083,5280,9
10,CS017414000114,5280,10


---
> P-021: レシート明細データセット（ds_receipt）に対し、件数をカウントせよ。

In [23]:
data _null_ ;
  set work.ds_receipt end = eof ;
  if eof = 1 then put "obs:" _n_ ;
run ;


51                                                        SAS システム               2021年 3月 7日 日曜日 12時53分00秒

394        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
394      ! ods graphics on / outputfmt=png;
395        
396        data _null_ ;
397          set work.ds_receipt end = eof ;
398          if eof = 1 then put "obs:" _n_ ;
399        run ;
obs:104681
400        
401        
402        ods html5 (id=saspy_internal) close;ods listing;
403        

52                                                        SAS システム               2021年 3月 7日 日曜日 12時53分00秒

404        


---
> P-022: レシート明細データセット（ds_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [24]:
proc sort data = work.ds_receipt(keep=customer_id) out = work._p_22 nodupkey ;
  by customer_id ;
run ;

data _null_ ;
  set  work._p_22 end = eof ;
  if eof = 1 then put "unique obs:" _n_ ;
run ;


53                                                        SAS システム               2021年 3月 7日 日曜日 12時53分00秒

407        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
407      ! ods graphics on / outputfmt=png;
408        
409        proc sort data = work.ds_receipt(keep=customer_id) out = work._p_22 nodupkey ;
410          by customer_id ;
411        run ;
412        
413        data _null_ ;
414          set  work._p_22 end = eof ;
415          if eof = 1 then put "unique obs:" _n_ ;
416        run ;
unique obs:8307
417        
418        
419        ods html5 (id=saspy_internal) close;ods listing;
420        

54                                                        SAS システム               2021年 3月 7日 日曜日 12時53分00秒

421        


---
> P-023: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [29]:
proc means data = work.ds_receipt nway noprint ;
  var amount quantity ;
  class store_cd ;
  output out = work._p_23
  sum = total_amount total_quantity 
  ;
run ;

proc print data = work._p_23 ;
  var store_cd total_amount total_quantity ;
run ;

Obs,store_cd,total_amount,total_quantity
1,S12007,638761,2099
2,S12013,787513,2425
3,S12014,725167,2358
4,S12029,794741,2555
5,S12030,684402,2403
6,S13001,811936,2347
7,S13002,727821,2340
8,S13003,764294,2197
9,S13004,779373,2390
10,S13005,629876,2004


---
> P-024: レシート明細データセット（ds_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [30]:
proc sort data = work.ds_receipt(keep=customer_id sales_ymd) out = work._p_24 ;
  by customer_id descending sales_ymd ;
run ;

data work._p_24 ;
  set  work._p_24 ;
  by customer_id ;
  if first.customer_id = 1 then ord = 0 ;
  ord + 1 ;
run ;

proc print data = work._p_24(obs=10) ; 
  var customer_id sales_ymd ;
  where ord = 1 ;
run ;

Obs,customer_id,sales_ymd
1,CS001113000004,20190308
3,CS001114000005,20190731
7,CS001115000010,20190405
13,CS001205000004,20190625
23,CS001205000006,20190224
35,CS001211000025,20190322
37,CS001212000027,20170127
39,CS001212000031,20180906
41,CS001212000046,20170811
43,CS001212000070,20191018


---
> P-025: レシート明細データセット（ds_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [31]:
proc sort data = work.ds_receipt(keep=customer_id sales_ymd) out = work._p_25 ;
  by customer_id sales_ymd ;
run ;

data work._p_25 ;
  set  work._p_25 ;
  by customer_id ;
  if first.customer_id = 1 then ord = 0 ;
  ord + 1 ;
run ;

proc print data = work._p_25(obs=10) ; 
  var customer_id sales_ymd ;
  where ord = 1 ;
run ;

Obs,customer_id,sales_ymd
1,CS001113000004,20190308
3,CS001114000005,20180503
7,CS001115000010,20171228
13,CS001205000004,20170914
23,CS001205000006,20180207
35,CS001211000025,20190322
37,CS001212000027,20170127
39,CS001212000031,20180906
41,CS001212000046,20170811
43,CS001212000070,20191018


---
> P-026: レシート明細データセット（ds_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [32]:
proc sort data = work.ds_receipt(keep=customer_id sales_ymd) out = work._p_26 ;
  by customer_id  sales_ymd ;
run ;

data work._p_26 ;
  set  work._p_26 ;
  by customer_id ;
  retain old ;
  if first.customer_id = 1 then old = sales_ymd ;
  if last.customer_id = 1 then do ;
    new = sales_ymd ;
    if old ^= new then output ;
  end ;
run ;

proc print data = work._p_26(obs=10) ; 
  var customer_id sales_ymd ;
run ;

Obs,customer_id,sales_ymd
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001214000009,20190902
6,CS001214000017,20191006
7,CS001214000048,20190929
8,CS001214000052,20190617
9,CS001215000005,20181021
10,CS001215000040,20171022


---
> P-027: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [33]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class store_cd ;
  output out = work._p_27  
         mean = mean 
         ;
run ;

proc sort data = work._p_27 ;  
  by descending mean ;
run ;

proc print data = work._p_27(obs=5) ;
  var store_cd mean ;
run ;

Obs,store_cd,mean
1,S13052,402.867
2,S13015,351.112
3,S13003,350.916
4,S14010,348.791
5,S13001,348.470


---
> P-028: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [34]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class store_cd ;
  output out = work._p_28  
         median = median 
         ;
run ;

proc sort data = work._p_28 ;  
  by descending median ;
run ;

proc print data = work._p_28(obs=5) ;
  var store_cd median ;
run ;

Obs,store_cd,median
1,S13052,190
2,S14010,188
3,S14050,185
4,S13003,180
5,S13018,180


---
> P-029: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [35]:
proc freq data = work.ds_receipt noprint ; 
  tables store_cd * product_cd / out = work._p_29 ;
run ;

proc sort data = work._p_29 ;
  by store_cd descending count ;
run ;

data work._p_29_mode ;
  set work._p_29 ;
  by store_cd ;
  if first.store_cd = 1 then output ;
run ;

proc print data = work._p_29_mode ;
  var store_cd product_cd count ;
run ;


Obs,store_cd,product_cd,COUNT
1,S12007,P060303001,72
2,S12013,P060303001,107
3,S12014,P060303001,65
4,S12029,P060303001,92
5,S12030,P060303001,115
6,S13001,P060303001,67
7,S13002,P060303001,78
8,S13003,P071401001,65
9,S13004,P060303001,88
10,S13005,P040503001,36


---
> P-030: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [36]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class store_cd ;
  output out = work._p_30  
         stdev = stdev 
         ;
run ;

proc sort data = work._p_30 ;  
  by descending stdev ;
run ;

data work._p_30 ;
  set work._p_30 ;
  variance = stdev **2 ;
run ;

proc print data = work._p_30(obs=5) ;
  var store_cd variance ;
run ;

Obs,store_cd,variance
1,S13052,441863.25
2,S14011,306442.24
3,S14034,297068.39
4,S13001,295558.84
5,S13015,295427.20


---
> P-031: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [37]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class store_cd ;
  output out = work._p_31  
         stdev = stdev 
         ;
run ;

proc sort data = work._p_31 ;  
  by descending stdev ;
run ;

proc print data = work._p_31(obs=5) ;
  var store_cd stdev ;
run ;

Obs,store_cd,stdev
1,S13052,664.728
2,S14011,553.572
3,S14034,545.040
4,S13001,543.653
5,S13015,543.532


---
> P-032: レシート明細データセット（ds_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [38]:
proc means data = work.ds_receipt min q1 median q3 max ; 
  var amount ;
run ;

---
> P-033: レシート明細データセット（ds_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [39]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class store_cd ;
  output out = work._p_33  
         mean = mean 
         ;
run ;

proc print data = work._p_33 ;
  var store_cd mean ;
  where mean >= 330 ;
run ;

Obs,store_cd,mean
2,S12013,330.194
6,S13001,348.470
8,S13003,350.916
9,S13004,330.944
13,S13015,351.112
17,S13019,330.209
18,S13020,337.880
29,S13052,402.867
31,S14010,348.791
32,S14011,335.718


---
> P-034: レシート明細データセット（ds_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [40]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_34  
         sum = sum 
         ;
run ;

proc means data = work._p_34 nway noprint ; 
  var sum ;
  output out = work._p_34_mean  
         mean = mean 
         ;
run ;

proc print data = work._p_34_mean ;
  var mean ;
run ;

Obs,mean
1,2547.74


---
> P-035: レシート明細データセット（ds_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [41]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_35  
         sum = sum 
         ;
run ;

data work._p_35 ;
  merge work._p_35
        work._p_34_mean
        ;
  retain _mean ;
  if _n_ = 1 then _mean = mean ;
run ;

proc print data = work._p_35(obs=10) ;
  var customer_id  sum ;
  where sum >= _mean ;
run ;

Obs,customer_id,sum
3,CS001115000010,3044
5,CS001205000006,3337
14,CS001214000009,4685
15,CS001214000017,4132
18,CS001214000052,5639
22,CS001215000040,3496
31,CS001304000006,3726
33,CS001305000005,3485
34,CS001305000011,4370
54,CS001315000180,3300


---
> P-036: レシート明細データセット（ds_receipt）と店舗データセット（ds_store）を内部結合し、レシート明細データセットの全項目と店舗データセットの店舗名（store_name）を10件表示させよ。

In [42]:
proc sort data = work.ds_receipt out = work._p_36 ;
  by store_cd ;
run ;

proc sort data = work.ds_store(keep=store_cd store_name) out = work._p_36_store ;
  by store_cd ;
run ;

data work._p_36 ;
  merge work._p_36(in=in1) 
        work._p_36_store(in=in2)
        ;
  by store_cd ;
  if in1 = 1 and in2 = 1 then output ;
run ;

proc print data = work._p_36(obs=10) ;
run ;

Obs,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
1,20190221,1550707200,S12007,1132,2,CS007513000235,P080104025,1,180,佐倉店
2,20170706,1499299200,S12007,1142,1,CS007615000066,P071401011,1,1300,佐倉店
3,20180904,1536019200,S12007,1112,1,CS007515000170,P060503001,1,78,佐倉店
4,20170909,1504915200,S12007,1102,2,ZZ000000000000,P070308044,1,548,佐倉店
5,20170623,1498176000,S12007,1182,1,CS007512000139,P060102005,1,90,佐倉店
6,20171130,1512000000,S12007,1162,1,ZZ000000000000,P091503004,1,1130,佐倉店
7,20170524,1495584000,S12007,1122,2,ZZ000000000000,P071005005,1,98,佐倉店
8,20170913,1505260800,S12007,1162,1,CS007414000032,P050601003,1,170,佐倉店
9,20181222,1545436800,S12007,1192,1,CS007514000086,P040102001,1,268,佐倉店
10,20180129,1517184000,S12007,1152,1,ZZ000000000000,P070503027,1,138,佐倉店


---
> P-037: 商品データセット（ds_product）とカテゴリデータセット（ds_category）を内部結合し、商品データセットの全項目とカテゴリデータセットの小区分名（category_small_name）を10件表示させよ。

In [43]:
proc sort data = work.ds_product out = work._p_37 ;
  by category_small_cd ;
run ;

proc sort data = work.ds_category(keep=category_small_cd category_small_name) out = work._p_37_name ;
  by category_small_cd ;
run ;

data work._p_37 ;
  merge work._p_37(in=in1) 
        work._p_37_name(in=in2)
        ;
  by category_small_cd ;
  if in1 = 1 and in2 = 1 then output ;
run ;

proc print data = work._p_37(obs=10) ;
run ;

Obs,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
1,P040101001,04,0401,040101,198,149,弁当類
2,P040101002,04,0401,040101,218,164,弁当類
3,P040101003,04,0401,040101,230,173,弁当類
4,P040101004,04,0401,040101,248,186,弁当類
5,P040101005,04,0401,040101,268,201,弁当類
6,P040101006,04,0401,040101,298,224,弁当類
7,P040101007,04,0401,040101,338,254,弁当類
8,P040101008,04,0401,040101,420,315,弁当類
9,P040101009,04,0401,040101,498,374,弁当類
10,P040101010,04,0401,040101,580,435,弁当類


---
> P-038: 顧客データセット（ds_customer）とレシート明細データセット（ds_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [45]:
proc sort data = work.ds_customer out = work._p_38 ;
  by customer_id ;
  where gender_cd = "1" and substr(customer_id, 1, 1) ^= "Z" ;
run ;

proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class customer_id ;
  output out = work._p_38_sum  
         sum = sum 
         ;
run ;

data work._p_38 ;
  merge work._p_38(in=in1) 
        work._p_38_sum
        ;
  by customer_id ;
  if sum = . then sum = 0 ;
  if in1 = 1 then output ;
run ;

proc print data = work._p_38(obs=10) ;
  var customer_id gender sum ;
run ;

Obs,customer_id,gender,sum
1,CS001112000009,女性,0
2,CS001112000019,女性,0
3,CS001112000021,女性,0
4,CS001112000023,女性,0
5,CS001112000024,女性,0
6,CS001112000029,女性,0
7,CS001112000030,女性,0
8,CS001113000004,女性,1298
9,CS001113000010,女性,0
10,CS001114000005,女性,626


---
> P-039: レシート明細データセット（ds_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [46]:
proc sort data = work.ds_receipt out = work._p_39 ;
  by customer_id sales_ymd ;
  where substr(customer_id, 1, 1) ^= "Z" ;
run ;

data work._p_39_day work._p_39_amount ;
  set work._p_39 ;
  by customer_id sales_ymd ;
  if first.customer_id = 1 then do ;
    day = 0 ;
    total = 0 ;
  end ;

  if first.sales_ymd = 1 then day + 1 ;
  total + amount ;

  if last.customer_id = 1 then output ;

  keep customer_id day total ;
run ;

proc sort data = work._p_39_day ;
  by descending day ;
run ;

proc sort data = work._p_39_day(obs = 20) out = work._p_39_day ;
  by customer_id ;
run ;

proc sort data = work._p_39_amount ;
  by descending total ;
run ;

proc sort data = work._p_39_amount(obs = 20) out = work._p_39_amount ;
  by customer_id ;
run ;

data work._p_39_d_a ;
  merge work._p_39_day 
        work._p_39_amount
        ;
  by customer_id ;
run ;


proc print data = work._p_39_d_a ;
run ;

Obs,customer_id,day,total
1,CS001605000009,9,18925
2,CS006515000023,15,18372
3,CS007514000094,13,15735
4,CS007515000107,18,11188
5,CS009414000059,17,15492
6,CS010214000002,21,13120
7,CS010214000010,22,18585
8,CS011414000106,16,18338
9,CS011415000006,11,16094
10,CS014214000023,19,8405


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（ds_store）と商品（ds_product）を直積した件数を計算せよ。

In [47]:
data _null_ ;
  set work.ds_store end = eof ;
  do i=1 to obs ;  
    set work.ds_product nobs=obs point=i ;
   /* output ;*/
    n + 1 ;
  end ;

  if eof = 1 then put "N:" n ;
run ;


99                                                        SAS システム               2021年 3月 7日 日曜日 12時53分00秒

992        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
992      ! ods graphics on / outputfmt=png;
993        
994        data _null_ ;
995          set work.ds_store end = eof ;
996          do i=1 to obs ;
997            set work.ds_product nobs=obs point=i ;
998           /* output ;*/
999            n + 1 ;
1000         end ;
1001       
1002         if eof = 1 then put "N:" n ;
1003       run ;
N:531590
1004       
1005       
1006       ods html5 (id=saspy_internal) close;ods listing;
1007       

100                                                       SAS システム               2021年 3月 7日 日曜日 12時53分00秒

1008       


---
> P-041: レシート明細データセット（ds_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [50]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class sales_ymd ;
  output out = work._p_41  
         sum = sum 
         ;
run ;

data work._p_41 ;
  set work._p_41 ;
  diff = sum - lag(sum) ;
run ;

proc print data = work._p_41(obs = 10) ;
  var sales_ymd sum diff ;
run ;

Obs,sales_ymd,sum,diff
1,20170101,33723,.
2,20170102,24165,-9558
3,20170103,27503,3338
4,20170104,36165,8662
5,20170105,37830,1665
6,20170106,32387,-5443
7,20170107,23415,-8972
8,20170108,24737,1322
9,20170109,26718,1981
10,20170110,20143,-6575


---
> P-042: レシート明細データセット（ds_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [52]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class sales_ymd ;
  output out = work._p_42  
         sum = sum 
         ;
run ;

data work._p_42 ;
  set work._p_42 ;
  diff1 = sum - lag(sum) ;
  diff2 = sum - lag2(sum) ;
  diff3 = sum - lag3(sum) ;

run ;

proc print data = work._p_42(obs = 10) ;
  var sales_ymd sum diff1 diff2 diff3 ;
run ;

Obs,sales_ymd,sum,diff1,diff2,diff3
1,20170101,33723,.,.,.
2,20170102,24165,-9558,.,.
3,20170103,27503,3338,-6220,.
4,20170104,36165,8662,12000,2442
5,20170105,37830,1665,10327,13665
6,20170106,32387,-5443,-3778,4884
7,20170107,23415,-8972,-14415,-12750
8,20170108,24737,1322,-7650,-13093
9,20170109,26718,1981,3303,-5669
10,20170110,20143,-6575,-4594,-3272


---
> P-043： レシート明細データセット（ds_receipt）と顧客データセット（ds_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータセット（ds_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [61]:
proc format ;
  value agegr
  0 - 9 = "0-9"
  10 - 19 = "10-19"
  20 - 29 = "20-29"
  30 - 39 = "30-39"
  40 - 49 = "40-49"
  50 - 59 = "50-59"
  60 - 69 = "60-69"
  70 - 79 = "70-79"
  80 - 89 = "80-89"
  90 - 99 = "90-99"
  ;
run ;

proc sort data = work.ds_receipt out = work._p_43 ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id gender: age) out = work._p_43_c ;
  by customer_id ;
run ;

data work._p_43 ;
  merge work._p_43(in=in1) 
        work._p_43_c
        ;
  by customer_id ;
  if in1 = 1 then output ;
run ;

proc means data = work._p_43 nway noprint ;
  var amount ;
  class age gender_cd gender ;
  format age agegr. ;
  output out = work._p_43_sum
        sum = sum
        ;
run ;

proc transpose data = work._p_43_sum out = work._p_43_t prefix = col_ ;
  var sum ;
  by age ;
  id gender_cd ;
  idlabel gender ;
run ;

proc print data = work._p_43_t(drop=_:) label ;
run ;

Obs,age,男性,女性,不明
1,10-19,1591,149836,4317
2,20-29,72940,1363724,44328
3,30-39,177322,693047,50441
4,40-49,19355,9320791,483512
5,50-59,54320,6685192,342923
6,60-69,272469,987741,71418
7,70-79,13435,29764,2427
8,80-89,46360,262923,5111
9,90-99,.,6260,.


---
> P-044： 前設問で作成した売上サマリデータセット（ds_sales_summary）は性別の売上を横持ちさせたものであった。このデータセットから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [66]:
proc transpose data = work._p_43_t out = work._p_44 ;
  var col: ;
  by age ;
run ;

data work._p_44 ;
  set work._p_44 ;
  _cd = scan(_name_, 2, "_") ;
  if _cd in("0","1") then pre = "0" ;
  else if _cd = "9" then pre = "9" ;
  gender_cd2 = cats(pre,_cd) ;
run ;

proc print data = work._p_44 ;
  var age gender_cd2 sum ;
run ;

Obs,age,gender_cd2,sum
1,10-19,00,1591
2,10-19,01,149836
3,10-19,99,4317
4,20-29,00,72940
5,20-29,01,1363724
6,20-29,99,44328
7,30-39,00,177322
8,30-39,01,693047
9,30-39,99,50441
10,40-49,00,19355


---
> P-045: 顧客データセット（ds_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [68]:
proc print data = work.ds_customer(obs=10) ;
  var customer_id birth_day ;
  format birth_day yymmddn8. ;
run ;

Obs,customer_id,birth_day
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502
10,CS033513000180,19620711


---
> P-046: 顧客データセット（ds_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [70]:
data work._p_46 ;
  set work.ds_customer ;
  applicationdt = input(application_date, yymmdd8.) ;
run ;

proc print data = work._p_46(obs=10) ;
  var customer_id applicationdt ;
run ;

Obs,customer_id,applicationdt
1,CS021313000114,20336
2,CS037613000071,20192
3,CS031415000172,20237
4,CS028811000001,20468
5,CS001215000145,20975
6,CS020401000016,20144
7,CS015414000103,20291
8,CS029403000008,20223
9,CS015804000004,20246
10,CS033513000180,20297


---
> P-047: レシート明細データセット（ds_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [72]:
data work._p_47 ;
  set work.ds_receipt ;
  salesdt = input(cats(sales_ymd), yymmdd8.) ;
run ;

proc print data = work._p_47(obs=10) ;
  var receipt_no receipt_sub_no salesdt ;
run ;

Obs,receipt_no,receipt_sub_no,salesdt
1,112,1,21491
2,1132,2,21506
3,1102,1,21012
4,1132,1,21585
5,1102,2,21417
6,1112,1,21705
7,1102,2,21523
8,1102,1,21814
9,1112,2,20943
10,1102,1,21832


---
> P-048: レシート明細データセット（ds_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [82]:
options TIMEZONE= 'JST' ;

data work._p_48 ;
  set work.ds_receipt ;
  epochdt = dhms('01jan1970'd,0,0, sales_epoch + tzoneoff()) ;
  epochdtc=put(epochdt,e8601dt.) ;
run ;

proc print data = work._p_48(obs=10) ;
  var receipt_no receipt_sub_no epochdt epochdtc ;
run ;

Obs,receipt_no,receipt_sub_no,epochdt,epochdtc
1,112,1,1856854800,2018-11-03T09:00:00
2,1132,2,1858150800,2018-11-18T09:00:00
3,1102,1,1815469200,2017-07-12T09:00:00
4,1132,1,1864976400,2019-02-05T09:00:00
5,1102,2,1850461200,2018-08-21T09:00:00
6,1112,1,1875344400,2019-06-05T09:00:00
7,1102,2,1859619600,2018-12-05T09:00:00
8,1102,1,1884762000,2019-09-22T09:00:00
9,1112,2,1809507600,2017-05-04T09:00:00
10,1102,1,1886317200,2019-10-10T09:00:00


---
> P-049: レシート明細データセット（ds_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [86]:
data work._p_49 ;
  set work.ds_receipt ;
  epochdt = dhms('01jan1970'd,0,0, sales_epoch + tzoneoff()) ;
  epochdtc=put(epochdt,e8601dt.) ;
  epochy = year(datepart(epochdt)) ;
run ;

proc print data = work._p_49(obs=10) ;
  var receipt_no receipt_sub_no epochy epochdtc ;
run ;

Obs,receipt_no,receipt_sub_no,epochy,epochdtc
1,112,1,2018,2018-11-03T09:00:00
2,1132,2,2018,2018-11-18T09:00:00
3,1102,1,2017,2017-07-12T09:00:00
4,1132,1,2019,2019-02-05T09:00:00
5,1102,2,2018,2018-08-21T09:00:00
6,1112,1,2019,2019-06-05T09:00:00
7,1102,2,2018,2018-12-05T09:00:00
8,1102,1,2019,2019-09-22T09:00:00
9,1112,2,2017,2017-05-04T09:00:00
10,1102,1,2019,2019-10-10T09:00:00


---
> P-050: レシート明細データセット（ds_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [88]:
data work._p_50 ;
  set work.ds_receipt ;
  epochdt = dhms('01jan1970'd,0,0, sales_epoch + tzoneoff()) ;
  epochdtc=put(epochdt,e8601dt.) ;
  epochm = put(month(datepart(epochdt)), z2.) ;
run ;

proc print data = work._p_50(obs=10) ;
  var receipt_no receipt_sub_no epochm epochdtc ;
run ;

Obs,receipt_no,receipt_sub_no,epochm,epochdtc
1,112,1,11,2018-11-03T09:00:00
2,1132,2,11,2018-11-18T09:00:00
3,1102,1,07,2017-07-12T09:00:00
4,1132,1,02,2019-02-05T09:00:00
5,1102,2,08,2018-08-21T09:00:00
6,1112,1,06,2019-06-05T09:00:00
7,1102,2,12,2018-12-05T09:00:00
8,1102,1,09,2019-09-22T09:00:00
9,1112,2,05,2017-05-04T09:00:00
10,1102,1,10,2019-10-10T09:00:00


---
> P-051: レシート明細データセット（ds_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [89]:
data work._p_51 ;
  set work.ds_receipt ;
  epochdt = dhms('01jan1970'd,0,0, sales_epoch + tzoneoff()) ;
  epochdtc=put(epochdt,e8601dt.) ;
  epochd = put(day(datepart(epochdt)), z2.) ;
run ;

proc print data = work._p_51(obs=10) ;
  var receipt_no receipt_sub_no epochd epochdtc ;
run ;

Obs,receipt_no,receipt_sub_no,epochd,epochdtc
1,112,1,03,2018-11-03T09:00:00
2,1132,2,18,2018-11-18T09:00:00
3,1102,1,12,2017-07-12T09:00:00
4,1132,1,05,2019-02-05T09:00:00
5,1102,2,21,2018-08-21T09:00:00
6,1112,1,05,2019-06-05T09:00:00
7,1102,2,05,2018-12-05T09:00:00
8,1102,1,22,2019-09-22T09:00:00
9,1112,2,04,2017-05-04T09:00:00
10,1102,1,10,2019-10-10T09:00:00


---
> P-052: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [91]:
proc means data = work.ds_receipt nway noprint ;
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_52
        sum = sum
        ;
run ;

data work._p_52 ;
  set work._p_52 ;
  flg = ifn(sum >2000, 1, 0) ;
run ;
  
proc print data = work._p_52(obs=10) ;
  var customer_id sum flg ;
run ;

Obs,customer_id,sum,flg
1,CS001113000004,1298,0
2,CS001114000005,626,0
3,CS001115000010,3044,1
4,CS001205000004,1988,0
5,CS001205000006,3337,1
6,CS001211000025,456,0
7,CS001212000027,448,0
8,CS001212000031,296,0
9,CS001212000046,228,0
10,CS001212000070,456,0


---
> P-053: 顧客データセット（ds_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データセット（ds_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [92]:
proc sort data = work.ds_receipt out = work._p_53 nodupkey ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id postal_cd) out = work._p_53_c ;
  by customer_id ;
run ;

data work._p_53 ;
  merge work._p_53(in=in1) 
        work._p_53_c
        ;
  by customer_id ;
  
  flg= ifn(input(substr(postal_cd, 1,3), best.) in (100:209), 1, 0) ; 
  if in1 = 1 then output ;
run ;

proc freq data = work._p_53 ;
  tables flg ;
run ;

flg,度数,パーセント,累積度数,累積パーセント
0,3907,47.03,3907,47.03
1,4400,52.97,8307,100.00


---
> P-054: 顧客データデータセット（ds_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [93]:
data work._p_54 ;  
  set work.ds_customer ;
  pref = kscan(address,1,"県都") ;
       if pref = "埼玉"   then prefcd = 11 ;
  else if pref = "千葉"   then prefcd = 12 ;
  else if pref = "東京"   then prefcd = 13 ;
  else if pref = "神奈川" then prefcd = 14 ;
run ;

proc print data = work._p_54(obs=10) ;
  var customer_id address prefcd ;
run ;

Obs,customer_id,address,prefcd
1,CS021313000114,神奈川県伊勢原市粟窪**********,14
2,CS037613000071,東京都江東区南砂**********,13
3,CS031415000172,東京都渋谷区代々木**********,13
4,CS028811000001,神奈川県横浜市泉区和泉町**********,14
5,CS001215000145,東京都大田区仲六郷**********,13
6,CS020401000016,東京都板橋区若木**********,13
7,CS015414000103,東京都江東区北砂**********,13
8,CS029403000008,千葉県浦安市海楽**********,12
9,CS015804000004,東京都江東区北砂**********,13
10,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [94]:
proc means data = work.ds_receipt nway noprint ;
  var amount ;
  class customer_id ;
  output out = work._p_55
        sum = sum
        ;
run ;

proc means data = work._p_55 nway noprint ;
  var sum ;
  output out = work._p_55_q
        q1 = _q1
        median = _q2
        q3 = _q3
        ;
run ;

proc format ;
  value sumcat
  1 = "最小値以上第一四分位未満"
  2 = "第一四分位以上第二四分位未満"
  3 = "第二四分位以上第三四分位未満"
  4 = "第三四分位以上"
  ;
run ;

data work._p_55 ;
  merge work._p_55
        work._p_55_q
        ;
  retain q1 q2 q3 ;
  if _n_ = 1 then do ;
    q1 = _q1 ;
    q2 = _q2 ;
    q3 = _q3 ;
  end ;
  
       if sum < q1 then sumcatn = 1 ;
  else if sum < q2 then sumcatn = 2 ;
  else if sum < q3 then sumcatn = 3 ;
  else                  sumcatn = 4 ;
  
  sumcat = put(sumcatn, sumcat.) ;
run ;

proc print data = work._p_55(obs = 10) ;
  var customer_id sum sumcatn sumcat ;
run ;

Obs,customer_id,sum,sumcatn,sumcat
1,CS001113000004,1298,2,第一四分位以上第二四分位未満
2,CS001114000005,626,2,第一四分位以上第二四分位未満
3,CS001115000010,3044,3,第二四分位以上第三四分位未満
4,CS001205000004,1988,3,第二四分位以上第三四分位未満
5,CS001205000006,3337,3,第二四分位以上第三四分位未満
6,CS001211000025,456,1,最小値以上第一四分位未満
7,CS001212000027,448,1,最小値以上第一四分位未満
8,CS001212000031,296,1,最小値以上第一四分位未満
9,CS001212000046,228,1,最小値以上第一四分位未満
10,CS001212000070,456,1,最小値以上第一四分位未満


---
> P-056: 顧客データセット（ds_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [95]:
proc format ;
  value agegr2_
  0 - 9 = "0-9"
  10 - 19 = "10-19"
  20 - 29 = "20-29"
  30 - 39 = "30-39"
  40 - 49 = "40-49"
  50 - 59 = "50-59"
  60 - high  = "60-"
  ;
run ;

proc print data = work.ds_customer(obs=10) ;
  var customer_id birth_day age ;
  format age agegr2_. ;
run ;

Obs,customer_id,birth_day,age
1,CS021313000114,1981-04-29,30-39
2,CS037613000071,1952-04-01,60-
3,CS031415000172,1976-10-04,40-49
4,CS028811000001,1933-03-27,60-
5,CS001215000145,1995-03-29,20-29
6,CS020401000016,1974-09-15,40-49
7,CS015414000103,1977-08-09,40-49
8,CS029403000008,1973-08-17,40-49
9,CS015804000004,1931-05-02,60-
10,CS033513000180,1962-07-11,50-59


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [97]:
 data work._p_57 ;
   set work.ds_customer ;
   genagecat = cats(gender, ":", put(age, agegr2_. )) ;
run ;

proc print data = work._p_57(obs=10) ;
  var customer_id birth_day genagecat ; 
run ;

Obs,customer_id,birth_day,genagecat
1,CS021313000114,1981-04-29,女性:30-39
2,CS037613000071,1952-04-01,不明:60-
3,CS031415000172,1976-10-04,女性:40-49
4,CS028811000001,1933-03-27,女性:60-
5,CS001215000145,1995-03-29,女性:20-29
6,CS020401000016,1974-09-15,男性:40-49
7,CS015414000103,1977-08-09,女性:40-49
8,CS029403000008,1973-08-17,男性:40-49
9,CS015804000004,1931-05-02,男性:60-
10,CS033513000180,1962-07-11,女性:50-59


---
> P-058: 顧客データセット（ds_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [98]:
 data work._p_58 ;
   set work.ds_customer ;
   flg0 = ifn(gender_cd=0, 1, 0) ;
   flg1 = ifn(gender_cd=1, 1, 0) ;
   flg9 = ifn(gender_cd=9, 1, 0) ;
run ;

proc print data = work._p_58(obs=10) ;
  var customer_id flg: ; 
run ;

Obs,customer_id,flg0,flg1,flg9
1,CS021313000114,0,1,0
2,CS037613000071,0,0,1
3,CS031415000172,0,1,0
4,CS028811000001,0,1,0
5,CS001215000145,0,1,0
6,CS020401000016,1,0,0
7,CS015414000103,0,1,0
8,CS029403000008,1,0,0
9,CS015804000004,1,0,0
10,CS033513000180,0,1,0


---
> P-059: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [103]:
proc means data = work.ds_receipt nway noprint ;
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_59
        sum = sum
        ;
run ;

proc standard data = work._p_59 out = work._p_59_score mean = 0 std = 1 noprint ;
  var sum ;
run ;

data work._p_59 ;
  merge work._p_59
        work._p_59_score(keep = sum rename=(sum=score))
        ;
run ;

proc print data = work._p_59(obs=10) ;
  var customer_id sum score ; 
run ;

Obs,customer_id,sum,score
1,CS001113000004,1298,-0.45935
2,CS001114000005,626,-0.70635
3,CS001115000010,3044,0.18240
4,CS001205000004,1988,-0.20574
5,CS001205000006,3337,0.29010
6,CS001211000025,456,-0.76883
7,CS001212000027,448,-0.77177
8,CS001212000031,296,-0.82764
9,CS001212000046,228,-0.85264
10,CS001212000070,456,-0.76883


---
> P-060: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [104]:
proc means data = work.ds_receipt nway noprint ;
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_60
        sum = sum
        ;
run ;

proc stdize data=work._p_60 out=work._p_60_scale method= range;
   var sum ;
run;

data work._p_60 ;
  merge work._p_60
        work._p_60_scale(keep = sum rename=(sum=scale))
        ;
run ;

proc print data = work._p_60(obs=10) ;
  var customer_id sum scale ; 
run ;

---
> P-061: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [105]:
proc means data = work.ds_receipt nway noprint ;
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_61
        sum = sum
        ;
run ;

data work._p_61 ;
  set work._p_61 ;
  log_sum = log10(sum) ;
run ;

proc print data = work._p_61(obs=10) ;
  var customer_id sum log_sum ; 
run ;

Obs,customer_id,sum,log_sum
1,CS001113000004,1298,3.11327
2,CS001114000005,626,2.79657
3,CS001115000010,3044,3.48344
4,CS001205000004,1988,3.29842
5,CS001205000006,3337,3.52336
6,CS001211000025,456,2.65896
7,CS001212000027,448,2.65128
8,CS001212000031,296,2.47129
9,CS001212000046,228,2.35793
10,CS001212000070,456,2.65896


---
> P-062: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [106]:
proc means data = work.ds_receipt nway noprint ;
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_62
        sum = sum
        ;
run ;

data work._p_62 ;
  set work._p_62 ;
  loge_sum = log(sum) ;
run ;

proc print data = work._p_62(obs=10) ;
  var customer_id sum loge_sum ; 
run ;

Obs,customer_id,sum,loge_sum
1,CS001113000004,1298,7.16858
2,CS001114000005,626,6.43935
3,CS001115000010,3044,8.02093
4,CS001205000004,1988,7.59488
5,CS001205000006,3337,8.11283
6,CS001211000025,456,6.12249
7,CS001212000027,448,6.10479
8,CS001212000031,296,5.69036
9,CS001212000046,228,5.42935
10,CS001212000070,456,6.12249


---
> P-063: 商品データセット（ds_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [108]:
data work._p_63 ;
  set work.ds_product ;
  if cmiss(unit_price, unit_cost) = 0 then profit = unit_price - unit_cost ;
run ;

proc print data = work._p_63(obs=10) ;
  var product_cd profit ; 
run ;

Obs,product_cd,profit
1,P040101001,49
2,P040101002,54
3,P040101003,57
4,P040101004,62
5,P040101005,67
6,P040101006,74
7,P040101007,84
8,P040101008,105
9,P040101009,124
10,P040101010,145


---
> P-064: 商品データセット（ds_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [110]:
data work._p_64 ;
  set work.ds_product ;
  if cmiss(unit_price, unit_cost) = 0 then profit_pct = (unit_price - unit_cost)/unit_cost * 100 ;
run ;

proc means data = work._p_64 nway mean nmiss ;
  var profit_pct ;
run ;

---
> P-065: 商品データセット（ds_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [113]:
data work._p_65 ;
  set work.ds_product ;
  if cmiss(unit_cost) = 0 then new_price = int(unit_cost*1.3) ;
  if cmiss(new_price, unit_cost) = 0 then new_pct = (new_price - unit_cost)/unit_cost * 100 ;
run ;

proc print data = work._p_65(obs=10) ;
  var product_cd new_price new_pct ;  
run ;

Obs,product_cd,new_price,new_pct
1,P040101001,193,29.5302
2,P040101002,213,29.8780
3,P040101003,224,29.4798
4,P040101004,241,29.5699
5,P040101005,261,29.8507
6,P040101006,291,29.9107
7,P040101007,330,29.9213
8,P040101008,409,29.8413
9,P040101009,486,29.9465
10,P040101010,565,29.8851


---
> P-066: 商品データセット（ds_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [114]:
data work._p_66 ;
  set work.ds_product ;
  if cmiss(unit_cost) = 0 then new_price = round(unit_cost*1.3) ;
  if cmiss(new_price, unit_cost) = 0 then new_pct = (new_price - unit_cost)/unit_cost * 100 ;
run ;

proc print data = work._p_66(obs=10) ;
  var product_cd new_price new_pct ;  
run ;

Obs,product_cd,new_price,new_pct
1,P040101001,194,30.2013
2,P040101002,213,29.8780
3,P040101003,225,30.0578
4,P040101004,242,30.1075
5,P040101005,261,29.8507
6,P040101006,291,29.9107
7,P040101007,330,29.9213
8,P040101008,410,30.1587
9,P040101009,486,29.9465
10,P040101010,566,30.1149


---
> P-067: 商品データセット（ds_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [123]:
data work._p_67 ;
  set work.ds_product ;
  if cmiss(unit_cost) = 0 then new_price = ceil(unit_cost*1.3) ;
  if cmiss(new_price, unit_cost) = 0 then new_pct = (new_price - unit_cost)/unit_cost * 100 ;
run ;

proc print data = work._p_67(obs=10) ;
  var product_cd new_price new_pct ;  
run ;

Obs,product_cd,new_price,new_pct
1,P040101001,194,30.2013
2,P040101002,214,30.4878
3,P040101003,225,30.0578
4,P040101004,242,30.1075
5,P040101005,262,30.3483
6,P040101006,292,30.3571
7,P040101007,331,30.3150
8,P040101008,410,30.1587
9,P040101009,487,30.2139
10,P040101010,566,30.1149


---
> P-068: 商品データセット（ds_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [124]:
data work._p_68 ;
  set work.ds_product ;
  if cmiss(unit_price) = 0 then tax_price = int(unit_price*1.1) ;
run ;

proc print data = work._p_68(obs=10) ;
  var product_cd tax_price  ;  
run ;

Obs,product_cd,tax_price
1,P040101001,217
2,P040101002,239
3,P040101003,253
4,P040101004,272
5,P040101005,294
6,P040101006,327
7,P040101007,371
8,P040101008,462
9,P040101009,547
10,P040101010,638


---
> P-069: レシート明細データセット（ds_receipt）と商品データセット（ds_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [127]:
proc sort data = work.ds_receipt out = work._p_69_re ;
  by product_cd ;
run ;

proc sort data = work.ds_product(keep=product_cd category_major_cd) out = work._p_69_pr ;
  by product_cd ;
run ;

data work._p_69_re_pr ;
  merge work._p_69_re
        work._p_69_pr
        ;
  by product_cd ;
  if category_major_cd = "07" then amount_07 = amount ;

run ;

proc means data = work._p_69_re_pr nway noprint ;
  var amount amount_07 ;
  class customer_id ;
  output out = work._p_69
        sum = sum sum07
        ;
run ;

data work._p_69 ;
  set work._p_69 ;
  if cmiss(sum07) = 0 then ratio= sum07/sum ;
run ;

proc print data = work._p_69(obs=10) ;
  var customer_id sum sum07 ratio ;
  where ratio ^= . ;
run ;

Obs,customer_id,sum,sum07,ratio
1,CS001113000004,1298,1298,1.00000
2,CS001114000005,626,486,0.77636
3,CS001115000010,3044,2694,0.88502
4,CS001205000004,1988,346,0.17404
5,CS001205000006,3337,2004,0.60054
7,CS001212000027,448,200,0.44643
8,CS001212000031,296,296,1.00000
9,CS001212000046,228,108,0.47368
10,CS001212000070,456,308,0.67544
12,CS001213000018,243,145,0.59671


---
> P-070: レシート明細データセット（ds_receipt）の売上日（sales_ymd）に対し、顧客データセット（ds_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [130]:
proc sort data = work.ds_receipt out = work._p_70_re ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id application_date) out = work._p_70_c ;
  by customer_id ;
run ;

data work._p_70 ;
  merge work._p_70_re(in=in1)
        work._p_70_c
        ;
  by customer_id ;
  if cmiss(sales_ymd, application_date) = 0 then durd = input(cats(sales_ymd),yymmdd10.) - input(application_date, yymmdd10.) ; 
  if dur > 0 then durd = durd + 1 ;
  
  if in1 = 1 then output ;
run ;

proc print data = work._p_70(obs=10) ;
  var customer_id sales_ymd application_date durd ;
run ;

Obs,customer_id,sales_ymd,application_date,dur
1,CS001113000004,20190308,20151105,1220
2,CS001113000004,20190308,20151105,1220
3,CS001114000005,20180503,20160412,752
4,CS001114000005,20180503,20160412,752
5,CS001114000005,20190731,20160412,1206
6,CS001114000005,20190731,20160412,1206
7,CS001115000010,20190405,20150417,1450
8,CS001115000010,20180701,20150417,1172
9,CS001115000010,20171228,20150417,987
10,CS001115000010,20190405,20150417,1450


---
> P-071: レシート明細データセット（ds_receipt）の売上日（sales_ymd）に対し、顧客データセット（ds_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [2]:
proc sort data = work.ds_receipt out = work._p_71_re ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id application_date) out = work._p_71_c ;
  by customer_id ;
run ;

data work._p_71 ;
  merge work._p_71_re(in=in1)
        work._p_71_c
        ;
  by customer_id ;
  if cmiss(sales_ymd, application_date)=0 then durm = intck('month', input(application_date, yymmdd10.), input(cats(sales_ymd),yymmdd10.), 'C')  ; 
  
  if in1 = 1 then output ;
run ;

proc print data = work._p_71(obs=10) ;
  var customer_id sales_ymd application_date durm ;
run ;

Obs,customer_id,sales_ymd,application_date,durm
1,CS001113000004,20190308,20151105,40
2,CS001113000004,20190308,20151105,40
3,CS001114000005,20180503,20160412,24
4,CS001114000005,20180503,20160412,24
5,CS001114000005,20190731,20160412,39
6,CS001114000005,20190731,20160412,39
7,CS001115000010,20190405,20150417,47
8,CS001115000010,20180701,20150417,38
9,CS001115000010,20171228,20150417,32
10,CS001115000010,20190405,20150417,47


---
> P-072: レシート明細データセット（ds_receipt）の売上日（sales_ymd）に対し、顧客データセット（ds_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [53]:
proc sort data = work.ds_receipt out = work._p_72_re ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id application_date) out = work._p_72_c ;
  by customer_id ;
run ;

data work._p_72 ;
  merge work._p_72_re(in=in1)
        work._p_72_c
        ;
  by customer_id ;
  if cmiss(sales_ymd, application_date)=0 then dury = intck('year', input(application_date, yymmdd10.), input(cats(sales_ymd),yymmdd10.), 'C')  ; 
  
  if in1 = 1 then output ;
run ;

proc print data = work._p_72(obs=10) ;
  var customer_id sales_ymd application_date dury ;
run ;

Obs,customer_id,sales_ymd,application_date,dury
1,CS001113000004,20190308,20151105,3
2,CS001113000004,20190308,20151105,3
3,CS001114000005,20180503,20160412,2
4,CS001114000005,20180503,20160412,2
5,CS001114000005,20190731,20160412,3
6,CS001114000005,20190731,20160412,3
7,CS001115000010,20190405,20150417,3
8,CS001115000010,20180701,20150417,3
9,CS001115000010,20171228,20150417,2
10,CS001115000010,20190405,20150417,3


---
> P-073: レシート明細データセット（ds_receipt）の売上日（sales_ymd）に対し、顧客データセット（ds_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [4]:
proc sort data = work.ds_receipt out = work._p_73_re ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id application_date) out = work._p_73_c ;
  by customer_id ;
run ;

data work._p_73 ;
  merge work._p_73_re(in=in1)
        work._p_73_c
        ;
  by customer_id ;
  if cmiss(sales_ymd, application_date)=0 then durs = dhms(input(cats(sales_ymd),yymmdd10.), 0, 0, 0) -  dhms(input(application_date, yymmdd10.), 0, 0, 0) ; 
  
  if in1 = 1 then output ;
run ;

proc print data = work._p_73(obs=10) ;
  var customer_id sales_ymd application_date durs ;
run ;

Obs,customer_id,sales_ymd,application_date,durs
1,CS001113000004,20190308,20151105,105321600
2,CS001113000004,20190308,20151105,105321600
3,CS001114000005,20180503,20160412,64886400
4,CS001114000005,20180503,20160412,64886400
5,CS001114000005,20190731,20160412,104112000
6,CS001114000005,20190731,20160412,104112000
7,CS001115000010,20190405,20150417,125193600
8,CS001115000010,20180701,20150417,101174400
9,CS001115000010,20171228,20150417,85190400
10,CS001115000010,20190405,20150417,125193600


---
> P-074: レシート明細データセット（ds_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [7]:
data work._p_74 ;
  set work.ds_receipt ;
  mondt = intnx("week.2", input(cats(sales_ymd),yymmdd10.), 0) ;
  durd = input(cats(sales_ymd),yymmdd10.) - mondt ;
run ;

proc print data = work._p_74(obs=10) ;
  var customer_id sales_ymd mondt durd ;
    format mondt yymmdd10. ;
run ;

Obs,customer_id,sales_ymd,mondt,durd
1,CS006214000001,20181103,2018-10-29,5
2,CS008415000097,20181118,2018-11-12,6
3,CS028414000014,20170712,2017-07-10,2
4,ZZ000000000000,20190205,2019-02-04,1
5,CS025415000050,20180821,2018-08-20,1
6,CS003515000195,20190605,2019-06-03,2
7,CS024514000042,20181205,2018-12-03,2
8,CS040415000178,20190922,2019-09-16,6
9,ZZ000000000000,20170504,2017-05-01,3
10,CS027514000015,20191010,2019-10-07,3


---
> P-075: 顧客データセット（ds_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [10]:
data work._p_75 ;
  set work.ds_customer ;
  _order = ranuni(12345) ;
run ;

proc sort data = work._p_75 ;
  by _order ;
run ;

data work._p_75 ;
  set work._p_75 nobs = obs ;
  if _n_ / obs <= 0.01 then output ;
  drop _order ;
run ;

proc print data = work._p_75(obs=10) ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS005715000130,岸部 千佳子,1,女性,1943-07-16,75,176-0021,東京都練馬区貫井**********,S13005,20170218,0-00000000-0
2,CS007615000076,吉原 美里,1,女性,1950-04-25,68,285-0845,千葉県佐倉市西志津**********,S12007,20150127,B-20100621-E
3,CS031603000002,竹田 賢治,0,男性,1955-03-25,64,151-0064,東京都渋谷区上原**********,S13031,20150319,0-00000000-0
4,CS014612000101,白川 幸子,1,女性,1958-01-30,61,263-0001,千葉県千葉市稲毛区長沼原町**********,S12014,20151130,0-00000000-0
5,CS033515000117,玉山 綾女,1,女性,1962-11-24,56,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150622,D-20100207-F
6,CS005411000112,筒井 あさみ,1,女性,1975-09-25,43,177-0031,東京都練馬区三原台**********,S13005,20170823,0-00000000-0
7,CS002315000397,福岡 彩華,1,女性,1980-01-03,39,185-0011,東京都国分寺市本多**********,S13002,20170125,0-00000000-0
8,CS028314000028,柴田 涼,1,女性,1983-11-16,35,246-0023,神奈川県横浜市瀬谷区阿久和東**********,S14028,20150924,3-20080219-5
9,CS039803000011,玉木 豊,0,男性,1930-07-20,88,167-0043,東京都杉並区上荻**********,S13039,20171229,0-00000000-0
10,CS038415000147,中村 寿々花,1,女性,1970-12-22,48,279-0042,千葉県浦安市東野**********,S13038,20150605,F-20100623-E


---
> P-076: 顧客データセット（ds_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [15]:
data work._p_76 ;
  set work.ds_customer ;
  _order = ranuni(12345) ;
run ;

proc sort data = work._p_76 ;
  by gender_cd _order ;
run ;

proc freq data = work.ds_customer noprint ;
  table gender_cd / out = work._count ;
run ;

data work._p_76 ;
  merge work._p_76 
        work._count
        ;
  by gender_cd ;
  
  retain gen_n ;
  
  if first.gender_cd = 1 then do ;
    gen_count = 0 ;
    gen_n = count ;
  end ;
  
  gen_count +1 ;
  
  if gen_count / gen_n <= 0.01 then output ;
  keep customer_id -- status_cd ;
  
run ;

proc print data = work._p_76(obs=10) ;
run ;

Obs,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS031603000002,竹田 賢治,0,男性,1955-03-25,64,151-0064,東京都渋谷区上原**********,S13031,20150319,0-00000000-0
2,CS039803000011,玉木 豊,0,男性,1930-07-20,88,167-0043,東京都杉並区上荻**********,S13039,20171229,0-00000000-0
3,CS002804000004,野島 兼,0,男性,1931-01-24,88,185-0021,東京都国分寺市南町**********,S13002,20170419,5-20091213-7
4,CS003603000048,石井 功補,0,男性,1950-08-18,68,201-0013,東京都狛江市元和泉**********,S13003,20160711,0-00000000-0
5,CS013402000023,伊集院 慎之介,0,男性,1969-09-18,49,275-0015,千葉県習志野市鷺沼台**********,S12013,20160118,0-00000000-0
6,CS032303000021,中原 直人,0,男性,1986-05-31,32,144-0055,東京都大田区仲六郷**********,S13032,20141113,0-00000000-0
7,CS039501000016,大原 慢太郎,0,男性,1965-08-05,53,168-0063,東京都杉並区和泉**********,S13039,20150330,0-00000000-0
8,CS043603000002,岡田 ひとり,0,男性,1955-08-10,63,140-0014,東京都品川区大井**********,S13043,20170624,0-00000000-0
9,CS008201000001,新井 優一,0,男性,1996-09-20,22,154-0021,東京都世田谷区豪徳寺**********,S13008,20151126,0-00000000-0
10,CS004404000008,山瀬 竜次,0,男性,1977-04-30,41,165-0027,東京都中野区野方**********,S13004,20150821,0-00000000-0


---
> P-077: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [9]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_77_sum  
         sum = sum
         ;
run ;

proc means data = work._p_77_sum nway noprint ; 
  var sum ;
  output out = work._p_77_std  
         stdev = std
         mean = mean
         ;
run ;

data work._p_77 ;
  merge work._p_77_sum
        work._p_77_std
        ;
  retain upper lower ;
  if _n_ = 1 then do ;
    upper = mean + std * 3 ;
    lower = mean - std * 3 ;
  end ;
  
  if upper <= sum or lower >= sum then output ;
run ;

proc print data = work._p_77(obs=10) ;
  var customer_id sum upper lower ;
run ;

Obs,customer_id,sum,upper,lower
1,CS001605000009,18925,10709.76,-5614.28
2,CS006415000147,12723,10709.76,-5614.28
3,CS006515000023,18372,10709.76,-5614.28
4,CS006515000125,12575,10709.76,-5614.28
5,CS006515000209,11373,10709.76,-5614.28
6,CS007115000006,11528,10709.76,-5614.28
7,CS007514000056,13293,10709.76,-5614.28
8,CS007514000094,15735,10709.76,-5614.28
9,CS007515000107,11188,10709.76,-5614.28
10,CS007615000026,11959,10709.76,-5614.28


---
> P-078: レシート明細データセット（ds_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [11]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class customer_id ;
  where substr(customer_id, 1, 1) ^= "Z" ;
  output out = work._p_78_sum  
         sum = sum
         ;
run ;

proc means data = work._p_78_sum nway noprint ; 
  var sum ;
  output out = work._p_78_std  
         stdev = std
         q1 = q1
         q3 = q3
         ;
run ;

data work._p_78 ;
  merge work._p_78_sum
        work._p_78_std
        ;
  retain upper lower ;
  if _n_ = 1 then do ;
    upper = q3 + (q3-q1) * 1.5 ;
    lower = q1 - (q3-q1) * 1.5 ;
  end ;
  
  if upper < sum or lower > sum then output ;
run ;

proc print data = work._p_78(obs=10) ;
  var customer_id sum upper lower ;
run ;

Obs,customer_id,sum,upper,lower
1,CS001414000048,8584,8303,-4105
2,CS001605000009,18925,8303,-4105
3,CS002415000594,9568,8303,-4105
4,CS004414000181,9584,8303,-4105
5,CS005415000137,8734,8303,-4105
6,CS006414000001,9156,8303,-4105
7,CS006414000029,9179,8303,-4105
8,CS006415000105,10042,8303,-4105
9,CS006415000147,12723,8303,-4105
10,CS006415000157,10648,8303,-4105


---
> P-079: 商品データセット（ds_product）の各項目に対し、欠損数を確認せよ。

In [15]:
proc format;
 value $miss ' '='Missing' other='Not Missing';
 value  miss  . ='Missing' other='Not Missing';
run;
 
proc freq data = work.ds_product ; 
  format _char_ $miss. ;
  tables _char_ / missing nocum nopct ;

  format _numeric_ miss. ;
  tables _numeric_ / missing nocum nopct;
run;

product_cd,度数
Not Missing,10030
category_major_cd,度数
Not Missing,10030
category_medium_cd,度数
Not Missing,10030
category_small_cd,度数
Not Missing,10030
unit_price,度数
Missing,7
Not Missing,10023


---
> P-080: 商品データセット（ds_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなds_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [23]:
data work._p_80 ;
  set work.ds_product end = eof ;
  where cmiss(product_cd, category_major_cd, category_medium_cd, category_small_cd, unit_price, unit_cost) = 0 ;
  if eof = 1 then put "obs:" _n_ ;
run ;


51                                                        SAS システム               2021年 3月14日 日曜日 15時37分00秒

777        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
777      ! ods graphics on / outputfmt=png;
778        
779        data work._p_80 ;
780          set work.ds_product end = eof ;
781          where cmiss(product_cd, category_major_cd, category_medium_cd, category_small_cd, unit_price, unit_cost) = 0 ;
782          if eof = 1 then put "obs:" _n_ ;
783        run ;
obs:10023
784        
785        
786        ods html5 (id=saspy_internal) close;ods listing;
787        

52                                                        SAS システム               2021年 3月14日 日曜日 15時37分00秒

788        


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなds_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [21]:
proc means data = work.ds_product nway noprint ; 
  var unit_price unit_cost ;
  output out = work._p_81  
         mean = _mean1 _mean2
         ;
run ;

data work.ds_product_2 ;
  merge work.ds_product end = eof 
        work._p_81
        ;
  retain mean1 mean2 ;
  if _n_ = 1 then do ;
    mean1 = round(_mean1) ;
    mean2 = round(_mean2) ;
  end ;
  
  unit_price = coalesce(unit_price, mean1) ;
  unit_cost = coalesce(unit_cost, mean2) ;
  
  if cmiss(of product_cd--unit_cost) >0 then miss_n + 1 ;
  
  if eof = 1 then put "missing:" miss_n ;
  keep product_cd--unit_cost ;
run ;



47                                                        SAS システム               2021年 3月14日 日曜日 15時37分00秒

728        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
728      ! ods graphics on / outputfmt=png;
729        
730        proc means data = work.ds_product nway noprint ;
731          var unit_price unit_cost ;
732          output out = work._p_81
733                 mean = _mean1 _mean2
734                 ;
735        run ;
736        
737        data work.ds_product_2 ;
738          merge work.ds_product end = eof
739                work._p_81
740                ;
741          retain mean1 mean2 ;
742          if _n_ = 1 then do ;
743            mean1 = _mean1 ;
744            mean2 = _mean2 ;
745          end ;
746        
747          unit_price = coalesce(unit_price, mean1) ;
748          unit_cost = coalesce(unit_cost, mean2) ;
749        
750          if cmiss(of product_cd--unit_cost) >0 then mis

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなds_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [25]:
proc means data = work.ds_product nway noprint ; 
  var unit_price unit_cost ;
  output out = work._p_82  
         median = _med1 _med2
         ;
run ;

data work.ds_product_3 ;
  merge work.ds_product end = eof 
        work._p_82
        ;
  retain med1 med2 ;
  if _n_ = 1 then do ;
    med1 = round(_med1) ;
    med2 = round(_med2) ;
  end ;
  
  unit_price = coalesce(unit_price, med1) ;
  unit_cost = coalesce(unit_cost, med2) ;
  
  if cmiss(of product_cd--unit_cost) >0 then miss_n + 1 ;  
  if eof = 1 then put "missing:" miss_n ;
  
  keep product_cd--unit_cost ;
  
run ;



55                                                        SAS システム               2021年 3月14日 日曜日 15時37分00秒

827        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
827      ! ods graphics on / outputfmt=png;
828        
829        proc means data = work.ds_product nway noprint ;
830          var unit_price unit_cost ;
831          output out = work._p_82
832                 median = _med1 _med2
833                 ;
834        run ;
835        
836        data work.ds_product_3 ;
837          merge work.ds_product end = eof
838                work._p_82
839                ;
840          retain med1 med2 ;
841          if _n_ = 1 then do ;
842            med1 = _med1 ;
843            med2 = _med2 ;
844          end ;
845        
846          unit_price = coalesce(unit_price, med1) ;
847          unit_cost = coalesce(unit_cost, med2) ;
848        
849          if cmiss(of product_cd--unit_cost) >0 then miss_n + 1 

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなds_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [26]:
proc means data = work.ds_product nway noprint ; 
  var unit_price unit_cost ;
  class category_small_cd ;
  output out = work._p_83  
         median = _med1 _med2
         ;
run ;

proc sort data = work.ds_product out = work.ds_product_3 ;
  by category_small_cd ;
run ;

data work.ds_product_3 ;
  merge work.ds_product_3 end = eof 
        work._p_83
        ;
  by category_small_cd ;
  
  retain med1 med2 ;
  if first.category_small_cd  = 1 then do ;
    med1 = round(_med1) ;
    med2 = round(_med2) ;
  end ;
  
  unit_price = coalesce(unit_price, med1) ;
  unit_cost = coalesce(unit_cost, med2) ;
  
  if cmiss(of product_cd--unit_cost) >0 then miss_n + 1 ;  
  if eof = 1 then put "missing:" miss_n ;
  
  keep product_cd--unit_cost ;
  
run ;



57                                                        SAS システム               2021年 3月14日 日曜日 15時37分00秒

863        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
863      ! ods graphics on / outputfmt=png;
864        
865        proc means data = work.ds_product nway noprint ;
866          var unit_price unit_cost ;
867          class category_small_cd ;
868          output out = work._p_83
869                 median = _med1 _med2
870                 ;
871        run ;
872        
873        proc sort data = work.ds_product out = work.ds_product_3 ;
874          by category_small_cd ;
875        run ;
876        
877        data work.ds_product_3 ;
878          merge work.ds_product_3 end = eof
879                work._p_83
880                ;
881          by category_small_cd ;
882        
883          retain med1 med2 ;
884          if first.category_small_cd  = 1 then do ;
885            med1 = _med1 ;
886

---
> P-084: 顧客データセット（ds_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [9]:
data work._p_84_re ;
  set  work.ds_receipt ;
  year = substr(cats(sales_ymd), 1, 4) ;
run ;

proc means data = work._p_84_re nway noprint ; 
  var amount ;
  class customer_id year ;
  output out = work._p_84_sum  
         sum = sum
         ;
run ;

proc transpose data = work._p_84_sum out = work._p_84_t prefix = year_ ;
  var sum ;
  id year ;
  by customer_id ;
run ;

proc sort data = work.ds_customer(keep=customer_id) out = work._p_84_c ;
  by customer_id ;
run ;

data work._p_84 ;
  merge work._p_84_c
        work._p_84_t end = eof
        ;
  by customer_id ;
   
  if year_2019 ^= . then ratio = year_2019 / sum(of year_:) ;
  else ratio = 0 ;

  if ratio = . then miss_n + 1 ;  
  if eof = 1 then put "missing:" miss_n ;
  
run ;

proc print data = work._p_84(obs=10) ;
  var customer_id ratio ;
run ;

Obs,customer_id,ratio
1,CS001105000001,0
2,CS001112000009,0
3,CS001112000019,0
4,CS001112000021,0
5,CS001112000023,0
6,CS001112000024,0
7,CS001112000029,0
8,CS001112000030,0
9,CS001113000004,1
10,CS001113000010,0


---
> P-085: 顧客データセット（ds_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データセット（ds_geocode）を紐付け、新たなds_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [54]:
proc means data = ds_geocode nway noprint ; 
  var longitude latitude ;
  class postal_cd ;
  output out = work._p_85_mean  
         sum = longitude_c latitude_c
         ;
run ;

proc sort data = work.ds_customer out = work._p_85_c ;
  by postal_cd ;
run ;

data work.ds_customer_1 ;
  merge work._p_85_c(in=in1)
        work._p_85_mean
        ;
  by postal_cd ;
   
  if in1 = 1 then output ;
  
run ;


39                                                        SAS システム               2021年 3月21日 日曜日 15時40分00秒

678        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
678      ! ods graphics on / outputfmt=png;
679        
680        proc means data = ds_geocode nway noprint ;
681          var longitude latitude ;
682          class postal_cd ;
683          output out = work._p_85_mean
684                 sum = longitude_c latitude_c
685                 ;
686        run ;
687        
688        proc sort data = work.ds_customer out = work._p_85_c ;
689          by postal_cd ;
690        run ;
691        
692        data work.ds_customer_1 ;
693          merge work._p_85_c(in=in1)
694                work._p_85_mean
695                ;
696          by postal_cd ;
697        
698          if in1 = 1 then output ;
699        
700        run ;
701        
702        
703        ods html5 (id=saspy_internal) close;ods l

---
> P-086: 前設問で作成した緯度経度つき顧客データセット（ds_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データセット（ds_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [55]:
proc sort data = work.ds_customer_1 out = work._p_86_c ;
  by application_store_cd ;
run ;

proc sort data = work.ds_store(keep= store_cd address longitude latitude) 
          out = work._p_86_s(rename=(store_cd=application_store_cd  address=address_s )) ;
  by store_cd ;
run ;


data work._p_86 ;
  merge work._p_86_c(in=in1)
        work._p_86_s
        ;
  by application_store_cd ;
  
  dist = geodist( latitude, longitude, latitude_c, longitude_c) ;
   
  if in1 = 1 then output ;
  
run ;

proc print data = work._p_86(obs=10) ;
  var customer_id address address_s dist ;
run ;

Obs,customer_id,address,address_s,dist
1,CS007214000024,東京都江東区東陽**********,千葉県佐倉市上志津,30.0785
2,CS007713000078,東京都品川区大井**********,千葉県佐倉市上志津,39.9111
3,CS007703000013,東京都渋谷区幡ヶ谷**********,千葉県佐倉市上志津,42.7895
4,CS007513000029,東京都世田谷区瀬田**********,千葉県佐倉市上志津,47.4599
5,CS007214000013,神奈川県藤沢市藤沢**********,千葉県佐倉市上志津,73.1728
6,CS007212000042,千葉県千葉市花見川区内山町**********,千葉県佐倉市上志津,3.0037
7,CS007212000006,千葉県千葉市花見川区内山町**********,千葉県佐倉市上志津,3.0037
8,CS007212000020,千葉県千葉市花見川区内山町**********,千葉県佐倉市上志津,3.0037
9,CS007212000044,千葉県千葉市花見川区内山町**********,千葉県佐倉市上志津,3.0037
10,CS007112000002,千葉県千葉市花見川区内山町**********,千葉県佐倉市上志津,3.0037


---
> P-087:  顧客データセット（ds_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データセット（ds_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [18]:
proc means data = work.ds_receipt nway noprint ; 
  var amount ;
  class customer_id ;
  output out = work._p_87_sum  
         sum = sum
         ;
run ;

proc sort data = work.ds_customer out = work._p_87_c ;
  by customer_id ;
run ;

data work._p_87_c ;
  merge work._p_87_c
        work._p_87_sum
        ;
  by customer_id ;
run ;

proc sort data =work._p_87_c ;
  by customer_name postal_cd descending sum customer_id ;
run ;

proc sort data = work._p_87_c out = work.ds_customer_u nodupkey ;
  by customer_name postal_cd ;
run ;


41                                                        SAS システム               2021年 3月21日 日曜日 09時25分00秒

740        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
740      ! ods graphics on / outputfmt=png;
741        
742        proc means data = work.ds_receipt nway noprint ;
743          var amount ;
744          class customer_id ;
745          output out = work._p_87_sum
746                 sum = sum
747                 ;
748        run ;
749        
750        proc sort data = work.ds_customer out = work._p_87_c ;
751          by customer_id ;
752        run ;
753        
754        data work._p_87_c ;
755          merge work._p_87_c
756                work._p_87_sum
757                ;
758          by customer_id ;
759        run ;
760        
761        proc sort data =work._p_87_c ;
762          by customer_name postal_cd descending sum customer_id ;
763        run ;
764        
765        proc sort d

---
> P-088: 前設問で作成したデータを元に、顧客データセットに統合名寄IDを付与したデータセット（ds_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [21]:
proc sort data = work.ds_customer out = work._p_88_c ;
  by customer_name postal_cd ;
run ;

proc sort data = work.ds_customer_u(keep=customer_id customer_name postal_cd) out = work._p_88_u(rename=(customer_id = new_id)) ;
  by customer_name postal_cd ; 
run ;

data work.ds_customer_n ;
  merge work._p_88_c
        work._p_88_u
        ;
  by customer_name postal_cd ;
run ;


47                                                        SAS システム               2021年 3月21日 日曜日 09時25分00秒

821        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
821      ! ods graphics on / outputfmt=png;
822        
823        proc sort data = work.ds_customer out = work._p_88_c ;
824          by customer_name postal_cd ;
825        run ;
826        
827        proc sort data = work.ds_customer_u(keep=customer_id customer_name postal_cd) out = work._p_88_u(rename=(customer_id =
827      ! new_id)) ;
828          by customer_name postal_cd ;
829        run ;
830        
831        data work.ds_customer_n ;
832          merge work._p_88_c
833                work._p_88_u
834                ;
835          by customer_name postal_cd ;
836        run ;
837        
838        
839        ods html5 (id=saspy_internal) close;ods listing;
840        

48                                                        SAS システム 

---
> P-閑話: ds_customer_1, ds_customer_nは使わないので削除する。

In [23]:
proc datasets lib = work noprint ;
  delete ds_customer_1 ds_customer_n ;
quit ;


51                                                        SAS システム               2021年 3月21日 日曜日 09時25分00秒

856        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
856      ! ods graphics on / outputfmt=png;
857        
858        proc datasets lib = work noprint ;
859          delete ds_customer_1 ds_customer_n ;
860        quit ;
861        
862        
863        ods html5 (id=saspy_internal) close;ods listing;
864        

52                                                        SAS システム               2021年 3月21日 日曜日 09時25分00秒

865        


---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [28]:
proc sort data = work.ds_receipt(keep=customer_id) out = work._p_89_r nodupkey ;
  by customer_id ;
run ;

proc sort data = work.ds_customer out = work._p_89_c ;
  by customer_id ;
run ;

data work._p_89 ;
  merge work._p_89_c
        work._p_89_r(in=in1)
        ;
  by customer_id ;
  _order = ranuni(12345) ;
  if in1 = 1 then output ;
run ;

proc sort data = work._p_89 ;
  by _order ;
run ;

data work._p_89_train
     work._p_89_test
     ;
  set work._p_89 nobs = obs ;

  if _n_ / obs < 0.8 then output work._p_89_train ;
  else output work._p_89_test ;
run ;


61                                                        SAS システム               2021年 3月21日 日曜日 09時25分00秒

1022       ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
1022     ! ods graphics on / outputfmt=png;
1023       
1024       proc sort data = work.ds_receipt(keep=customer_id) out = work._p_89_r nodupkey ;
1025         by customer_id ;
1026       run ;
1027       
1028       proc sort data = work.ds_customer out = work._p_89_c ;
1029         by customer_id ;
1030       run ;
1031       
1032       data work._p_89 ;
1033         merge work._p_89_c
1034               work._p_89_r(in=in1)
1035               ;
1036         by customer_id ;
1037         _order = ranuni(12345) ;
1038         if in1 = 1 then output ;
1039       run ;
1040       
1041       proc sort data = work._p_89 ;
1042         by _order ;
1043       run ;
1044       
1045       data work._p_89_train
1046            work._p_89_test
1047        

---
> P-090: レシート明細データセット（ds_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [32]:
data work._p_90_re ;
  set  work.ds_receipt ;
  year = substr(cats(sales_ymd), 1, 4) ;
  month = substr(cats(sales_ymd), 5, 2) ;
run ;

proc means data = work._p_90_re nway noprint ; 
  var amount ;
  class year month ;
  output out = work._p_90_sum  
         sum = sum
         ;
run ;

%macro make_ds ;
  %do i = 1 %to 3 ;

data work._p_90_sum ;
  set  work._p_90_sum ;
  _order = ranuni(&i.) ;
run ;

proc sort data = work._p_90_sum ;
  by _order ;
run ;

data work._p_90_train_&i.
     work._p_90_test_&i.
     ;
  set work._p_90_sum ;
  
       if  1 <= _n_ <= 12 then output work._p_90_train_&i. ;
  else if 13 <= _n_ <= 18 then output work._p_90_test_&i. ;

run ;
%end ;
%mend ;

%make_ds ;


69                                                        SAS システム               2021年 3月21日 日曜日 09時25分00秒

1210       ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
1210     ! ods graphics on / outputfmt=png;
1211       
1212       data work._p_90_re ;
1213         set  work.ds_receipt ;
1214         year = substr(cats(sales_ymd), 1, 4) ;
1215         month = substr(cats(sales_ymd), 5, 2) ;
1216       run ;
1217       
1218       proc means data = work._p_90_re nway noprint ;
1219         var amount ;
1220         class year month ;
1221         output out = work._p_90_sum
1222                sum = sum
1223                ;
1224       run ;
1225       
1226       %macro make_ds ;
1227         %do i = 1 %to 3 ;
1228       
1229       data work._p_90_sum ;
1230         set  work._p_90_sum ;
1231         _order = ranuni(&i.) ;
1232       run ;
1233       
1234       proc sort data = work._p_90_sum ;
1235         by 

---
> P-091: 顧客データセット（ds_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [61]:
proc sort data = work.ds_receipt(keep=customer_id) out = work._p_91_r nodupkey ;
  by customer_id ;
run ;

proc sort data = work.ds_customer out = work._p_91_c ;
  by customer_id ;
run ;

data work._p_91_all ;
  merge work._p_91_c
        work._p_91_r(in = in1)
        ;
  by customer_id ;
  _order = ranuni(12345) ;
  flg = in1 ;
run ;

proc sort data = work._p_91_all ;
  by flg _order ;
run ;

data work._p_91_all ;
  set work._p_91_all ;
  if flg = 1 then n1 + 1 ;
  else n2 + 1 ;
  
  if flg= 1 then _order2 = n1 ;
  else _order2 = n2 ;
run ;

proc sort data = work._p_91_all ;
  by _order2  ;
run ;

data work._p_91 ;
  set work._p_91_all ;
  by _order2 ;
  if first._order2 = 0 or last._order2 = 0 then output ;
run ;


53                                                        SAS システム               2021年 3月21日 日曜日 15時40分00秒

875        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
875      ! ods graphics on / outputfmt=png;
876        
877        proc sort data = work.ds_receipt(keep=customer_id) out = work._p_91_r nodupkey ;
878          by customer_id ;
879        run ;
880        
881        proc sort data = work.ds_customer out = work._p_91_c ;
882          by customer_id ;
883        run ;
884        
885        data work._p_91_all ;
886          merge work._p_91_c
887                work._p_91_r(in = in1)
888                ;
889          by customer_id ;
890          _order = ranuni(12345) ;
891          flg = in1 ;
892        run ;
893        
894        proc sort data = work._p_91_all ;
895          by flg _order ;
896        run ;
897        
898        data work._p_91_all ;
899          set work._p_91_all ;
900      

---
> P-092: 顧客データセット（ds_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [39]:
data work._p_92_std ;
  set work.ds_customer ;
  drop gender ;
run ;

proc sort data = work.ds_customer(keep=gender_cd gender) out = work._p_92_code nodupkey ;
  by gender_cd gender ;
run ;


9                                                         SAS システム               2021年 3月21日 日曜日 15時40分00秒

109        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
109      ! ods graphics on / outputfmt=png;
110        
111        data work._p_92_std ;
112          set work.ds_customer ;
113          drop gender ;
114        run ;
115        
116        proc sort data = work.ds_customer(keep=gender_cd gender) out = work._p_92_code nodupkey ;
117          by gender_cd gender ;
118        run ;
119        
120        
121        ods html5 (id=saspy_internal) close;ods listing;
122        

10                                                        SAS システム               2021年 3月21日 日曜日 15時40分00秒

123        


---
> P-093: 商品データセット（ds_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータセット（ds_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データセットを作成せよ。

In [2]:
proc sort data = work.ds_product out = work._p_93_p ;
  by category_major_cd category_medium_cd category_small_cd ;
run ;

proc sort data = work.ds_category out = work._p_93_ca ;
  by category_major_cd category_medium_cd category_small_cd ;
run ;

data work._p_93 ;
  merge work._p_93_p(in = in1)
        work._p_93_ca
        ;
  by category_major_cd category_medium_cd category_small_cd ;
  if in1 = 1 then output ;
run ;


9                                                         SAS システム               2021年 3月23日 火曜日 15時44分00秒

102        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
102      ! ods graphics on / outputfmt=png;
103        
104        proc sort data = work.ds_product out = work._p_93_p ;
105          by category_major_cd category_medium_cd category_small_cd ;
106        run ;
107        
108        proc sort data = work.ds_category out = work._p_93_ca ;
109          by category_major_cd category_medium_cd category_small_cd ;
110        run ;
111        
112        data work._p_93 ;
113          merge work._p_93_p(in = in1)
114                work._p_93_ca
115                ;
116          by category_major_cd category_medium_cd category_small_cd ;
117          if in1 = 1 then output ;
118        run ;
119        
120        
121        ods html5 (id=saspy_internal) close;ods listing;
122        

10                 

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [7]:
filename out "&dir./ds_product2.csv" encoding = "utf-8" ;

proc export data = work._p_93 
  outfile= out
  dbms=csv replace ;
run ;


19                                                        SAS システム               2021年 3月23日 火曜日 15時44分00秒

262        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
262      ! ods graphics on / outputfmt=png;
263        
264        filename out "&dir./ds_product2.csv" encoding = "utf-8" ;
265        
266        proc export data = work._p_93
267          outfile= out
268          dbms=csv replace ;
269        run ;
NOTE: Unable to open parameter catalog: SASUSER.PARMS.PARMS.SLIST in update mode. Temporary parameter values will be saved to 
WORK.PARMS.PARMS.SLIST.
270         /**********************************************************************
271         *   PRODUCT:   SAS
272         *   VERSION:   9.4
273         *   CREATOR:   External File Interface
274         *   DATE:      23MAR21
275         *   DESC:      Generated SAS Datastep Code
276         *   TEMPLATE SOURCE:  (None Specified.)
277         ******

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [8]:
filename out "&dir./ds_product3.csv" encoding = "cp932" ;

proc export data = work._p_93 
  outfile= out
  dbms=csv replace ;
run ;


21                                                        SAS システム               2021年 3月23日 火曜日 15時44分00秒

337        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
337      ! ods graphics on / outputfmt=png;
338        
339        filename out "&dir./ds_product3.csv" encoding = "cp932" ;
340        
341        proc export data = work._p_93
342          outfile= out
343          dbms=csv replace ;
344        run ;
NOTE: Unable to open parameter catalog: SASUSER.PARMS.PARMS.SLIST in update mode. Temporary parameter values will be saved to 
WORK.PARMS.PARMS.SLIST.
345         /**********************************************************************
346         *   PRODUCT:   SAS
347         *   VERSION:   9.4
348         *   CREATOR:   External File Interface
349         *   DATE:      23MAR21
350         *   DESC:      Generated SAS Datastep Code
351         *   TEMPLATE SOURCE:  (None Specified.)
352         ******

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [9]:
filename out "&dir./ds_product4.csv" encoding = "utf-8" ;

proc export data = work._p_93 
  outfile= out
  dbms=csv replace ;
  putnames=no ;
run ;


23                                                        SAS システム               2021年 3月23日 火曜日 15時44分00秒

412        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
412      ! ods graphics on / outputfmt=png;
413        
414        filename out "&dir./ds_product4.csv" encoding = "utf-8" ;
415        
416        proc export data = work._p_93
417          outfile= out
418          dbms=csv replace ;
419          putnames=no ;
420        run ;
NOTE: Unable to open parameter catalog: SASUSER.PARMS.PARMS.SLIST in update mode. Temporary parameter values will be saved to 
WORK.PARMS.PARMS.SLIST.
421         /**********************************************************************
422         *   PRODUCT:   SAS
423         *   VERSION:   9.4
424         *   CREATOR:   External File Interface
425         *   DATE:      23MAR21
426         *   DESC:      Generated SAS Datastep Code
427         *   TEMPLATE SOURCE:  (None Spe

---
> P-097: 先に作成した以下形式のファイルを読み込み、データセットを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [10]:
filename in_data "&dir./ds_product2.csv" encoding = "utf-8" ;

proc import out = work._p_97
    datafile= in_data
    dbms = csv replace ;
    getnames = yes ;
    datarow = 2 ;
run ;

proc print data = work._p_97(obs=10) ;
run ;

Obs,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類
10,P040101010,4,401,40101,580,435,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データセットを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [12]:
filename in_data "&dir./ds_product4.csv" encoding = "utf-8" ;

proc import out = work._p_98
    datafile= in_data
    dbms = csv replace ;
    getnames = no ;
run ;

proc print data = work._p_98(obs=10) ;
run ;

Obs,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類
10,P040101010,4,401,40101,580,435,惣菜,御飯類,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [14]:
filename out "&dir./ds_product5.csv" encoding = "utf-8" ;

proc export data = work._p_93
  outfile= out
  dbms=tab replace ;
run ;


33                                                        SAS システム               2021年 3月23日 火曜日 15時44分00秒

665        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
665      ! ods graphics on / outputfmt=png;
666        
667        filename out "&dir./ds_product5.csv" encoding = "utf-8" ;
668        
669        proc export data = work._p_93
670          outfile= out
671          dbms=tab replace ;
672        run ;
NOTE: Unable to open parameter catalog: SASUSER.PARMS.PARMS.SLIST in update mode. Temporary parameter values will be saved to 
WORK.PARMS.PARMS.SLIST.
673         /**********************************************************************
674         *   PRODUCT:   SAS
675         *   VERSION:   9.4
676         *   CREATOR:   External File Interface
677         *   DATE:      23MAR21
678         *   DESC:      Generated SAS Datastep Code
679         *   TEMPLATE SOURCE:  (None Specified.)
680         ******

---
> P-100: 先に作成した以下形式のファイルを読み込み、データセットを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [15]:
filename in_data "&dir./ds_product5.csv" encoding = "utf-8" ;

proc import out = work._p_100
    datafile= in_data
    dbms = tab replace ;
    getnames = yes ;
    datarow = 2 ;
run ;

proc print data = work._p_100(obs=10) ;
run ;

Obs,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類
10,P040101010,4,401,40101,580,435,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！